In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from astropy.table import Table
import astropy.io.fits as fits

In [2]:
from glob import glob
import os

In [3]:
rows = []
for fn in glob("../nights_fixed/n[1-5]/*.fit"):
    d = dict(fits.getheader(fn))
    d.update({"fn":os.path.basename(fn)})
    rows.append(d)

In [4]:
df = pd.DataFrame(rows)

In [5]:
deletecolumns = ['BITPIX', 'BSCALE', 'BZERO', 'CCDBIN1', 'CCDBIN2', 'COMMENT',
                 'GAIN', 'TIMESYS', 'TELESCOP', 'SECPIX1', 'SECPIX2', 'SIMPLE',
                 'FILTPOS', 'FILTER', 'EXTEND', 'OBSERVAT', 'OBSERVER', 'INSTRUME']
for key in deletecolumns: del df[key]

In [6]:
df.columns

Index(['AIRMASS', 'CCDPICNO', 'DATE-OBS', 'DEC', 'EQUINOX', 'EXPTIME',
       'GPROBE', 'HA', 'IMAGETYP', 'IRAFNAME', 'JD', 'LAMPS', 'LATITUDE',
       'LONGITUD', 'MIRROR', 'NAXIS', 'NAXIS1', 'NAXIS2', 'OBJECT', 'RA',
       'ROTANGLE', 'ST', 'TELFOCUS', 'TIME-OBS', 'fn'],
      dtype='object')

In [7]:
import re

In [8]:
# NOTE: MT19 was actually not for us.
def flagobject(OBJECT):
    if bool(re.match("10-[\d]+", OBJECT)):
        return 'g10'
    elif bool(re.match("[0-9]+-[0-9]+", OBJECT)):
        return 'obj'
    elif re.match("Bias|Flat|Hg-Ne-Ne|HgNe\+Ne", OBJECT):
        return 'calib'
    elif re.match("HIP[0-9]+|MS[0-9]+|G[0-9]+|HD[0-9]+|BD[+\d]+|MT[\d]+", OBJECT):
        return 'calibstar'
    else:
        raise ValueError("unknown object type {:s}".format(OBJECT))

In [9]:
objtype = list(map(flagobject, df.OBJECT))
df['objtype'] = objtype

In [11]:
df.objtype.value_counts()

obj          685
calib        149
calibstar    132
g10           51
Name: objtype, dtype: int64

In [12]:
df.to_csv("../data/observed.csv")